# District by Section

In [ ]:
%load_ext sql
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
%%sql --save report_districts
SELECT 
    r.district as district,
    r.assigned_section as section,
    COUNT(*) count
FROM cts_forms_report r
WHERE district notnull
GROUP BY district, section
HAVING COUNT(*) > 1
ORDER BY count desc

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

plt.style.use('_mpl-gallery')

plt.rcParams['figure.figsize'] = [4, 4]

def make_district_chart(section='(any)'):
    results = %sql --with report_districts SELECT * \
        FROM report_districts \
        WHERE \
        '{{section}}' = '(any)' \
        OR ('{{section}}' != '(any)' AND section='{{section}}');
    df = results.DataFrame()
    count = list(df['count'])
    districts = list(df['district'])
    y = list(districts)
    x = list(count)
    fig, ax = plt.subplots()
    ax.barh(y, x)
    ax.set_xlabel('Number of Reports')
    ax.set_ylabel('District')
    ax.set_title(f'Reports by District for: {section}')
    return plt


In [ ]:
sections = %sql SELECT section FROM report_districts GROUP BY 1;
sections = sections.DataFrame().section

for section in sections:
    district_chart = make_district_chart(section)
    district_chart.show()
